# Create clusters using 60 PCA components (95% variance explained)
1. Create clusters 
    - KMeans
    - Allgo
2. Index
3. Create

In [39]:
# To show plot in line and stop auto scroll
%matplotlib inline

In [40]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [41]:
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy.stats import gaussian_kde
from scipy import stats
from scipy.spatial.distance import cdist, pdist, euclidean
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import collections

In [42]:
# Get train and test data 
train = pd.read_csv('./data/train_120feats_Dense_OutlierFree_LogTransform.csv')
test = pd.read_csv('./data/test_119feats_Dense_OutlierFree_LogTransform.csv')

In [51]:
# This function returns the dataframe in clusters.
def putClusterIndex(df, nCluster = 2, method = "KMeans", splitFile = False, writeToFile = False, outPut = 'Train'):
    
    # PCA stage
    print("Perform PCA..")
        # Copy salePrice and save it somewhere else
    if outPut == "Train":
        salePriceCol = df['SalePrice'].copy()
        df.drop(['SalePrice'], axis = 1, inplace = True)
        #  Scale the rest cols
    elif outPut == "Test":
        pass
    else:
        print("You have to choose either 'Test' or 'Train' as output.")
    trainMatrix = df.as_matrix()
    trainMatrix = preprocessing.scale(trainMatrix)
    dfPCA = PCA(n_components=60).fit_transform(trainMatrix) 
    
    # Clustering stage
    print("Use {0} to perform clustering..".format(method))
        # Perform clustering 
    if method == 'KMeans':
        clusterResult = KMeans(n_clusters=nCluster, init='k-means++', n_init=50, max_iter=2000).fit(dfPCA)
    if method == 'Agglo':
        clusterResult = AgglomerativeClustering(n_clusters=nCluster).fit(dfPCA) 
        # Add cluster indexes to the dataframe
    df['Clusterlabel'] = clusterResult.labels_
        # Print out the clusters and items count in each cluster
    clusterStats = collections.Counter(clusterResult.labels_)
    
    # Print out results
    print("Clustering Results:")
    for key, value in clusterStats.items():
        print("There is {0} items in Cluster #{1}".format(key, value))
    
    # Data output
        # Add SalePrice back to the dataframe
        if outPut == "Train":
            df['SalePrice'] = salePriceCol
        # When NOT split files according on cluster index:
    if splitFile == False and writeToFile == True:
        print("Save ONE file to local...")
        print("Save to ./data/FinalRR{0}_{1}_Dense_OF_NoTransform.csv".format(method, outPut))
        df.to_csv("./data/FinalRR{0}_{1}_Clustered_Dense_OF_Log.csv".format(method, outPut),index=False)
        # When DO SPLIT files according to cluster index:
    if splitFile == True and writeToFile == True:
        print("Save MULTIPLE files to local...")
        for cluster in np.unique(clusterResult.labels_):
            temp = df[df['Clusterlabel']==cluster].copy()
            temp.drop(['Clusterlabel'], axis = 1, inplace = True)
            print("Save to ./data/FinalRR{0}_Cluster{1}_{2}_Dense_OF_NoTransform.csv".format(method, cluster, outPut))
            temp.to_csv("./data/FinalRR{0}_Cluster{1}_{2}_Dense_OF_NoTransform.csv".format(method, cluster, outPut),index=False)
    print("Program Finished!")

In [54]:
putClusterIndex(test,nCluster = 2, method = "KMeans",splitFile = False, writeToFile = True, outPut = 'Test')

Perform PCA..
Use KMeans to perform clustering..
Clustering Results:
There is 1 items in Cluster #851
There is 0 items in Cluster #608
Save ONE file to local...
Save to ./data/FinalRRKMeans_Test_Dense_OF_NoTransform.csv
Program Finished!
